> <b>Note: This Jupyter Notebook is associated with the article: [How To Trade Options with Alpaca's Dashboard and Trading API](https://alpaca.markets/learn/how-to-trade-options-with-alpaca).</b>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/options/options-trade-options-with-alpaca.ipynb)

## Options Trading Using Alpaca-py

### 1. Setup

In [ ]:
# 1.1 Install / Update Alpaca-py
!pip install --upgrade alpaca-py

In [ ]:
# 1.2 Import Necessary Packages
import json
from datetime import datetime, timedelta
import os

import pandas as pd
from dotenv import load_dotenv
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.option import OptionDataStream

from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    GetAssetsRequest,
    MarketOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest,
    OptionLegRequest
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderClass,
    OrderType,
    TimeInForce,
    QueryOrderStatus,
    ContractType
)
from alpaca.common.exceptions import APIError

In [ ]:
# Please safely store your API keys and never commit them to the repository (use .gitignore)
# Load environment variables from environment file (e.g. .env)
load_dotenv()

TRADE_API_KEY = os.environ.get('ALPACA_API_KEY')
TRADE_API_SECRET = os.environ.get('ALPACA_SECRET_KEY')

# We use paper environment for this example
ALPACA_PAPER_TRADE = True

# Below are the variables for development this documents so please do not change these variables
TRADE_API_WSS = None
DATA_API_URL = None
OPTION_STREAM_DATA_WSS = None

### 2. Setup clients

Load API keys securely from a .env file\
You can get them from https://alpaca.markets/

* Step 1: Create a file named `.env` in your project root (same folder as this notebook)
* Step 2: Add your paper (live) trading keys in this format:
    * ALPACA_API_KEY=your_actual_api_key
    * ALPACA_SECRET_KEY=your_actual_secret_key
* Step 3: Make sure you have the 'python-dotenv' package installed:
    `!pip install python-dotenv`

In [ ]:
# To enable live trading, replace `ALPACA_PAPER_TRADE = True` with `ALPACA_PAPER_TRADE = False` to disable the paper trading configuration
trade_client = TradingClient(api_key=TRADE_API_KEY, secret_key=TRADE_API_SECRET, paper=ALPACA_PAPER_TRADE)
stock_data_client = StockHistoricalDataClient(api_key=TRADE_API_KEY, secret_key=TRADE_API_SECRET)

### 3. Check trading account

In [ ]:
# Check account related information
acc = trade_client.get_account()
acc

### 4. Check account configuration

In [ ]:
# Check account configurations
acct_config = trade_client.get_account_configurations()
acct_config

### 5. Retrieve list of assets which are options enabled

In [ ]:
# We can filter assets by `options_enabled` attribute
# Asset object has `options_enabled` attribute if it is options enabled
req = GetAssetsRequest(
  attributes = "options_enabled"  
)
assets = trade_client.get_all_assets(req)
assets

### 6. Get list of options contracts for the given symbol (e.g. SPY)

In [ ]:
# In this instance, we retrieve the necessary contracts based on the parameters we've configured.
# specify underlying symbol

underlying_symbol = "SPY"
req = GetOptionContractsRequest(
    underlying_symbol=[underlying_symbol],  
    # specify asset status: active (default)
    status=AssetStatus.ACTIVE,   
    # specify expiration date (specified date + 1 day range)
    expiration_date = None,
    # we can pass a date object or string (YYYY-MM-DD) to specify an expiration date range
    expiration_date_gte = None,                            
    expiration_date_lte = None, 
    # specify root symbol
    root_symbol = None,
    # specify option type (ContractType.CALL or ContractType.PUT)
    type = None,  
)

res = trade_client.get_option_contracts(req)
res

### 7. Get options contract by symbol

In [ ]:
symbol = res.option_contracts[0].symbol
contract = trade_client.get_option_contract(symbol)
contract

### 8. Get options contract by id

In [ ]:
contract_id = res.option_contracts[0].id
contract = trade_client.get_option_contract(symbol_or_id=contract_id)
contract

### 9. Get put options contracts
Please note that the stock SPY is used as an example and should not be considered investment advice.

In [ ]:
# We can filter the contracts that are relevant to us by specifying various option properties like expiration date range (the contracts which are expiring within a particular time window), strike range (the contracts with strike prices within a given range), style of expiration for the option contract (like American or European), etc.

underlying_symbol = "SPY"

# specify expiration date range
now = datetime.now(tz=ZoneInfo("America/New_York"))
day1 = now + timedelta(days=1)
day60 = now + timedelta(days=60)

req = GetOptionContractsRequest(
    # specify underlying symbol
    underlying_symbol = [underlying_symbol],
    # specify asset status: active (default)
    status = AssetStatus.ACTIVE,
    # specify expiration date (specified date + 1 day range)
    expiration_date = None,
    # we can pass date object or string
    expiration_date_gte = day1.date(),
    expiration_date_lte = day60.strftime(format="%Y-%m-%d"),
    # specify root symbol
    root_symbol = None,
    # specify option type: put
    type = ContractType.PUT,    
    # specify option style: american
    style = ExerciseStyle.AMERICAN,
    # specify strike price range
    strike_price_gte = None,
    strike_price_lte = None,
    # specify limit
    limit = 100,
    # specify page
    page = None
)

res = trade_client.get_option_contracts(req)
res

### 10. Get the highest open interest contract

In [ ]:
# Filter contract with the highest open interest

max_open_interest = 0
high_open_interest_contract = None
for contract in res.option_contracts:
    oi = contract.open_interest
    if oi is not None:
        try:
            oi_int = int(oi)
            if oi_int > max_open_interest:
                max_open_interest = oi_int
                high_open_interest_contract = contract
        # skip if not convertible to int
        except ValueError:
            continue 

### 11. Explore information about the selected contract

In [ ]:
# Check contract information like expiry, expiration style and contract status

#Check option expiry
print(high_open_interest_contract.expiration_date)

#Check option status
print(high_open_interest_contract.status)

#Check option exercise style
print(high_open_interest_contract.style)

### 12. Place a long put option order

**NOTE:**
* On expiration days, option orders must be submitted before 3:15 p.m. ET and before 3:30 p.m. ET for broad-based ETFs like SPY and QQQ. Orders placed after these times will be rejected.
* Expiring positions are auto-liquidated starting at 3:30 p.m. ET (options) and 3:45 p.m. ET (broad-based ETFs) for risk management.

In [ ]:
# Execute either a Buy Call or Put option order in accordance with your trading strategy

req = MarketOrderRequest(
    symbol=high_open_interest_contract.symbol,
    qty=1,
    side=OrderSide.BUY,
    type=OrderType.MARKET,
    time_in_force=TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

### 13. Get list of requested orders by specifying option contract symbol

In [ ]:
req = GetOrdersRequest(
    status=QueryOrderStatus.ALL,
    symbols=[high_open_interest_contract.symbol],
    limit=2,
)
orders = trade_client.get_orders(req)
orders

### 14. Filter order with the desired client_order_id and print the status

The `client_order_id` parameter allows traders to specify their own identifier when submitting an order, enabling them to manage the order using their own reference

In [ ]:
import uuid
client_order_id = f"order_{uuid.uuid4().hex[:8]}"

# Execute a Buy Call or Put option order with a custom client_order_id for order tracking and identification
req = MarketOrderRequest(
    symbol=high_open_interest_contract.symbol,
    qty=1,
    side=OrderSide.BUY,
    type=OrderType.MARKET,
    time_in_force=TimeInForce.DAY,
    client_order_id=client_order_id,
)
res = trade_client.submit_order(req)
res

In [ ]:
# Extract order information by the custom client_order_id
trade_client.get_order_by_client_id(client_id=client_order_id)

### 15. Get positions filtered by option contract symbol

In [ ]:
# 15.1 Get all positions
positions = trade_client.get_all_positions()
positions

In [ ]:
#15.2 Get open positions by symbol
positions = trade_client.get_open_position(
symbol_or_asset_id=high_open_interest_contract.symbol
)
positions

In [ ]:
# 15.3 Get open positions by contract id
positions = trade_client.get_open_position(
symbol_or_asset_id=high_open_interest_contract.id
)
positions

In [ ]:
# 15.4 check cost basis and unrealized profit/loss
# Show cost basis
print(positions.cost_basis)

# Show unrealized profit/loss
print(positions.unrealized_pl)

### 16. Close the option position

In [ ]:
trade_client.close_position(
    symbol_or_asset_id=high_open_interest_contract.symbol,
    close_options=ClosePositionRequest(qty="1")
)

### 17. Trade Update (Stream)

In [ ]:
# This allows the event loop to support `asyncio.run()` inside the notebook.
# !pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Retrieve real-time live trade update data from the moment the order is placed until execution.
# Subscribe trade updates

trade_stream_client = TradingStream(TRADE_API_KEY, TRADE_API_SECRET, paper=ALPACA_PAPER_TRADE, url_override=TRADE_API_WSS)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

### 18. Market Data (Historical)

In [ ]:
# setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(TRADE_API_KEY, TRADE_API_SECRET, url_override=DATA_API_URL)

# get option latest quote by symbol
req = OptionLatestQuoteRequest(
    symbol_or_symbols=[high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_quote(req)

In [ ]:
# get option latest trade by symbol
req = OptionLatestTradeRequest(
    symbol_or_symbols=[high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_trade(req)

In [ ]:
# get option chain by underlying_symbol
req = OptionChainRequest(
    underlying_symbol=high_open_interest_contract.underlying_symbol,
)
option_historical_data_client.get_option_chain(req)

### 19. Market Data (Stream)

In [ ]:
# This allows the event loop to support `asyncio.run()` inside the notebook.
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Live streaming of market data based on a collection of symbols or an individual symbol
option_data_stream_client = OptionDataStream(TRADE_API_KEY, TRADE_API_SECRET, url_override=OPTION_STREAM_DATA_WSS)

async def option_data_stream_handler(data):
    print(data)

symbols=[
    high_open_interest_contract.symbol,
]

option_data_stream_client.subscribe_quotes(option_data_stream_handler, *symbols) 
option_data_stream_client.subscribe_trades(option_data_stream_handler, *symbols)

option_data_stream_client.run()

### 20. Place Multi-Leg Options Position

**Note**: 
* The stock SPY is used as an example and should not be considered investment advice.
* The example below utilizes one of the multi-leg option strategies called a long straddle.

In [ ]:
# 20.1 Get the underlying stock price and define the strike range to narrow down the option chain.

# Define a underlying symbol we want to trade
underlying_symbol = 'SPY'

# Get the latest price of the underlying stock
def get_underlying_price(symbol):
    underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=symbol)
    underlying_trade_response = stock_data_client.get_stock_latest_trade(underlying_trade_request)
    return underlying_trade_response[symbol].price

# Get the latest price of the underlying stock
underlying_price = get_underlying_price(underlying_symbol)

# Define a 1% range around the underlying price 
STRIKE_RANGE = 0.01

# Set the minimum and maximum strike prices based on the underlying price
min_strike = str(underlying_price * (1 - STRIKE_RANGE))
max_strike = str(underlying_price * (1 + STRIKE_RANGE))

print(f"{underlying_symbol} price: {underlying_price}")

In [ ]:
# 20.2 Define a function to find near at-the-money (ATM) options

# This is a function that will return a contract which minimizes the difference from a target price
def find_nearest_strike_contract(contracts, target_price):
    min_diff = 0
    min_contract = None
    for contract in contracts:
        diff = abs(float(contract.strike_price) - target_price)
        if min_contract is None or diff < min_diff:
            min_diff = diff
            min_contract = contract
    return min_contract

In [ ]:
# 20.3 Narrow down the options to near-ATM options

# Obtain both call options and put options of the specified underlying asset
underlying_symbol = ['SPY']
order_legs = []

for c_type in [ContractType.CALL, ContractType.PUT]:
    print(c_type)
    req = GetOptionContractsRequest(
        underlying_symbols=underlying_symbol,
        status=AssetStatus.ACTIVE,
        expiration_date_gte = now.date() + timedelta(days=7),
        expiration_date_lte = now.date() + timedelta(days=8),
        style=ExerciseStyle.AMERICAN,
        strike_price_gte=min_strike,
        strike_price_lte=max_strike,
        type=c_type,
        page_token=None
    )
    # Get option chain of the underlying symbol for both contract types
    cts = trade_client.get_option_contracts(req)

    c = find_nearest_strike_contract(cts.option_contracts, underlying_price)
    if c is not None:
        order_legs.append(OptionLegRequest(
            symbol=c.symbol,
            side=OrderSide.BUY,
            ratio_qty=1
        ))
    else:
        print(f"No suitable contract found for {c_type}")

# We should see that the symbols are similar, like "SPY______C________" and "SPY______P________",
# with all values marked as "_" being the same in both symbols.
# Such is because we expect only the contract type (call or put, C or P) to be different.

order_legs

In [ ]:
# 20.4 Place the order for both legs simultaneously to execute the long straddle

# Set up an order request
req = MarketOrderRequest(
    qty=1,
    order_class=OrderClass.MLEG,
    time_in_force=TimeInForce.DAY,
    legs=order_legs
)
# place the order of the long straddle
res = trade_client.submit_order(req)
res